## Intro

Polars is a popular single machine DataFrame library powered by an OLAP Query Engine. Beginning in the v1.3 release, Polars can now leverage NVIDIA GPUs for even higher performance through its GPU engine (powered by RAPIDS cuDF).

This new engine is built directly into the Polars Lazy API – just pass <font color="#76B900">**engine="gpu"**</font> to the `collect` operation.

# Setup

## Installing the Polars GPU Engine

Install Polars with GPU Engine by using a feature flag in the standard pip install command  `pip install polars[gpu] --extra-index-url=https://pypi.nvidia.com`.

In [13]:
!pip install polars==1.5
!pip install "/content/cudf_polars_cu11-24.8.0a433-py3-none-any.whl" --extra-index-url=https://pypi.anaconda.org/rapidsai-wheels-nightly/simple

Looking in indexes: https://pypi.org/simple, https://pypi.anaconda.org/rapidsai-wheels-nightly/simple
Processing ./cudf_polars_cu11-24.8.0a433-py3-none-any.whl
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 18.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.7/18.7 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.9 MB/s eta 0:00:00
  Created wheel for cudf-cu11: filename=cudf_cu11-24.8.2-cp310-cp310-manylinux_2_28_x86_64.whl size=508918902 sha256=6e5bddab29fa53150e52048f560d4d538fad5ce39d71e746ef88aa8fa745

## Download the Data

We'll be working with a roughly 22GB dataset of [simulated financial transactions from Kaggle](https://www.kaggle.com/datasets/conorsully1/simulated-transactions) by default. If you're running this notebook on Google Colab using the T4 GPU in the Colab free tier, make sure you set `GOOGLE_COLAB=True` below to download a smaller version of this dataset (about 20% of the size) to fit on the relatively weaker CPU and GPU.

We're downloading a copy of this dataset from a GCS bucket hosted by NVIDIA to provide faster download speeds. We'll start by downloading the data. This should take about 30 seconds.

In [14]:
!wget https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet -O transactions.parquet

--2024-09-16 21:48:41--  https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.175.207, 74.125.24.207, 142.251.10.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.175.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 857658422 (818M) [application/octet-stream]
Saving to: ‘transactions.parquet’

transactions.parque 100%[===================>] 817.93M  22.8MB/s    in 38s     

2024-09-16 21:49:20 (21.4 MB/s) - ‘transactions.parquet’ saved [857658422/857658422]



# Getting Started

In [15]:
import polars as pl
from polars.testing import assert_frame_equal

In [16]:
transactions = pl.scan_parquet("transactions.parquet")

In [17]:
transactions.collect_schema()

Schema([('CUST_ID', String),
        ('START_DATE', Date),
        ('END_DATE', Date),
        ('TRANS_ID', String),
        ('DATE', Date),
        ('YEAR', Int64),
        ('MONTH', Int64),
        ('DAY', Int64),
        ('EXP_TYPE', String),
        ('AMOUNT', Float64)])

In [18]:
transactions.head(5).collect()

CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT
str,date,date,str,date,i64,i64,i64,str,f64
"""CI6XLYUMQK""",2015-05-01,null,"""T8I9ZB5A6X90UG8""",2015-09-11,2015,9,11,"""Motor/Travel""",20.27
"""CI6XLYUMQK""",2015-05-01,null,"""TZ4JSLS7SC7FO9H""",2017-02-08,2017,2,8,"""Motor/Travel""",12.85
"""CI6XLYUMQK""",2015-05-01,null,"""TTUKRDDJ6B6F42H""",2015-08-01,2015,8,1,"""Housing""",383.8
"""CI6XLYUMQK""",2015-05-01,null,"""TDUHFRUKGPPI6HD""",2019-03-16,2019,3,16,"""Entertainment""",5.72
"""CI6XLYUMQK""",2015-05-01,null,"""T0JBZHBMSVRFMMD""",2015-05-15,2015,5,15,"""Entertainment""",11.06


First operation: Total aggregate transaction amount

In [19]:
%%time
transactions.select(pl.col("AMOUNT").sum()).collect()

CPU times: user 1.45 s, sys: 358 ms, total: 1.81 s
Wall time: 1.08 s


AMOUNT
f64
3.6183e9


In [20]:
%%time
transactions.select(pl.col("AMOUNT").sum()).collect(engine="gpu")

CPU times: user 2.15 s, sys: 662 ms, total: 2.82 s
Wall time: 3.76 s


AMOUNT
f64
3.6183e9


In [21]:
transactions.collect().shape

(52393943, 10)

In [22]:
gpu_engine = pl.GPUEngine(
    device=0,
    raise_on_fail=True,
)

The very first collection on the GPU will take a couple of seconds. The GPU engine is lazy-loaded so that even if the necessary packages are installed, Polars' fast import times are not affected. Consequently, when we trigger GPU execution for the first time, we load a number of additional packages, and initialize GPU-specific data structures and contexts.

In [ ]:
%%time
transactions.select(pl.col("AMOUNT").sum()).collect(engine=gpu_engine)

CPU times: user 298 ms, sys: 30.4 ms, total: 329 ms
Wall time: 394 ms


AMOUNT
f64
3.6183e9


# More Complex Analysis

While the data is synthetic, it's representative of the kinds of datasets that come up in financial services, retail/e-commerce, consumer internet, and other industries.

With this data, we can see how using GPU-accelerated Polars provides a significant productivity boosts by exploring common business questions like which customers have the largest total transctions?

## Which customers have the largest total transactions?

In [ ]:
%%time

high_spenders_cpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect()
)
high_spenders_cpu

CPU times: user 4.27 s, sys: 2.46 s, total: 6.73 s
Wall time: 4.41 s


CUST_ID,AMOUNT
str,f64
"""CA9UYOQ5DA""",2.0290e6
"""CJUK2MTM5Q""",1.8115e6
"""CYXX1NBIKL""",1.8082e6
"""C6ILEYAYQ9""",1.7961e6
"""CCNBC305GI""",1.7274e6


In [ ]:
%%time

high_spenders_gpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect(engine=gpu_engine)
)
high_spenders_gpu

CPU times: user 345 ms, sys: 40.5 ms, total: 385 ms
Wall time: 397 ms


CUST_ID,AMOUNT
str,f64
"""CA9UYOQ5DA""",2.0290e6
"""CJUK2MTM5Q""",1.8115e6
"""CYXX1NBIKL""",1.8082e6
"""C6ILEYAYQ9""",1.7961e6
"""CCNBC305GI""",1.7274e6


In addition to the Dataframe interface, Polars also has an SQL interface. We can also use this with the GPU engine, since Polars translates both the DataFrame and SQL interfaces into a query execution plan.

In [ ]:
sql_query = """
SELECT CUST_ID, SUM(AMOUNT) as sum_amt
FROM transactions
GROUP BY CUST_ID
ORDER BY sum_amt desc
LIMIT 5
"""

%time pl.sql(sql_query).collect()
%time pl.sql(sql_query).collect(engine=gpu_engine)

CPU times: user 4.39 s, sys: 2.5 s, total: 6.89 s
Wall time: 4.53 s
CPU times: user 336 ms, sys: 39.2 ms, total: 376 ms
Wall time: 356 ms


CUST_ID,sum_amt
str,f64
"""CA9UYOQ5DA""",2.0290e6
"""CJUK2MTM5Q""",1.8115e6
"""CYXX1NBIKL""",1.8082e6
"""C6ILEYAYQ9""",1.7961e6
"""CCNBC305GI""",1.7274e6
